In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.append("../")
import housinginsights
from housinginsights.ingestion import ManifestReader, DataReader
from housinginsights.ingestion import ACSRentCleaner, GenericCleaner
from housinginsights.ingestion import load_meta_data
import json


ACS cleaner loaded


In [3]:
meta = load_meta_data('meta_sample.json')
manifest = ManifestReader('manifest_sample.csv')

In [4]:
from housinginsights.ingestion import CSVWriter

from housinginsights.tools import dbtools
from housinginsights.ingestion import DataSql


#temporary, to test pandas implementation version
from sqlalchemy import create_engine
db_string = dbtools.get_connect_str("local_database")
engine = create_engine(db_string)

#temporary, cursor instead
#cursor = dbtools.get_psycopg2_cursor("local_database")

#actual, use connection
db_conn = dbtools.get_database_connection("local_database")
dbapi_conn = db_conn.connection
cursor = dbapi_conn.cursor()


In [5]:

for manifest_row in manifest:
    if manifest_row['include_flag'] == 'use': #temp - see load_data for better version
        csv_reader = DataReader(manifest_row=manifest_row)
        csv_writer = CSVWriter(meta = meta, manifest_row = manifest_row)

        sql_writer = DataSql(meta, manifest_row, db_conn)

        #Assign an appropriate testing cleaner
        if manifest_row['destination_table'] == "acs_rent_median_temp":
            cleaner = ACSRentCleaner(meta, manifest_row)
        else:
            cleaner = GenericCleaner(meta,manifest_row)

        for idx, data_row in enumerate(csv_reader):
            clean_data_row = cleaner.clean(data_row, idx)
            if clean_data_row != None:
                csv_writer.write(clean_data_row)
            
        csv_writer.close()
        sql_writer.create_table()
        sql_writer.read_and_write_to_sql()
    
db_conn.close()
print(db_conn)

ready to append
tried to commit changes


In [6]:
db_conn = dbtools.get_database_connection("local_database")
result = db_conn.execute("SELECT * FROM acs_rent_median_temp")

for r in result:
    print(r)


('1400000US11001000100', '11001000100', 'Census Tract 1, District of Columbia, District of Columbia', 1980, '375')
('1400000US11001000201', '11001000201', 'Census Tract 2.01, District of Columbia, District of Columbia', 0, '**')
('1400000US11001001500', '11001001500', 'Census Tract 15, District of Columbia, District of Columbia', 3500, '***')
('1400000US11001002701', '11001002701', 'Census Tract 27.01, District of Columbia, District of Columbia', 1270, '111')
('1400000US11001004902', '11001004902', 'Census Tract 49.02, District of Columbia, District of Columbia', 1386, '491')
('1400000US11001007504', '11001007504', 'Census Tract 75.04, District of Columbia, District of Columbia', 778, '56')
